In [9]:
from pyspark import SparkContext, SparkConf
import pandas as pd

In [10]:
data_csv = pd.read_csv('predict_position.csv')

In [11]:
data_csv.head()

,Unnamed: 0,grid,position,constructor_standings_position,driver_standings_position
0,0,1,1,3,5
1,1,3,5,3,12
2,2,5,2,6,13
3,3,7,3,7,7
4,4,13,6,7,20


In [12]:
# print name of all heads
print(data_csv.columns)

Index(['Unnamed: 0', 'grid', 'position', 'constructor_standings_position',
       'driver_standings_position'],
      dtype='object')


In [13]:
# drop Unnamed: 0 column
data_csv = data_csv.drop('Unnamed: 0', axis=1)

In [15]:
data_csv.head()
# save
data_csv.to_csv('predict_position.csv', index=False)

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# create a SparkSession
spark = SparkSession.builder.appName("LogisticRegressionExample").getOrCreate()

# load data from a CSV file
data = spark.read.csv("predict_position.csv", header=True, inferSchema=True)

# select the columns to use in the model
feature_cols = ["grid", "constructor_standings_position", "driver_standings_position"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(data)

# select the label column
data = data.select("features", "position")

# split the data into training and testing sets
train_data, test_data = data.randomSplit([0.7, 0.3], seed=123)

# create a logistic regression model
lr = LogisticRegression(featuresCol="features", labelCol="position")

# train the model on the training data
lr_model = lr.fit(train_data)

# evaluate the model on the testing data
eval_result = lr_model.evaluate(test_data)

# print the accuracy of the model
print("Accuracy: ", eval_result.accuracy)
# print f1
print("F1: ", eval_result.weightedFMeasure())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/15 19:29:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/15 19:29:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Accuracy:  0.16134689582602596
F1:  0.13646532331922887
